**Deep Convolutional Neural Network CNN for Sentiment Analysis**

The Movie Review Data est une collection de critiques de films extraites du site Web imdb.com au début des années 2000 par Bo Pang et Lillian Lee. Les revues ont été rassemblées et mises à disposition dans le cadre de leurs recherches sur le traitement du langage naturel.

Composer de: 
- 1 000 critiques de films positives 
- 1 000 critiques de films négatives 

tirées des archives du groupe de discussion rec.arts.movies.reviews hébergé sur imdb.com.

In [143]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
# turn a doc into clean tokens
def clean_doc(doc, vocab):
    tokens = doc.split()
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

def process_docs(directory, vocab,is_trian):
    documents = list()
    for filename in listdir(directory):
        if is_trian and filename.startswith('cv9'):
            continue
        if not is_trian and not filename.startswith('cv9'):
            continue
        path = directory + '/' + filename
        doc = load_doc(path)
        tokens = clean_doc(doc, vocab)
        documents.append(tokens)
    return documents

def save_list(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

save_list(tokens, 'vocab.txt')
negat_docs = process_docs('txt_sentoken/neg', vocab,True)
posit_docs = process_docs('txt_sentoken/pos', vocab,True)
train_docs = negat_docs + posit_docs


In [144]:
#Train
##code les documents d'apprentissage sous forme de  séquences d'entiers 
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)

max_length = max([len(s.split()) for s in train_docs])

Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])


In [145]:
#Test
positive_docs = process_docs('txt_sentoken/pos', vocab, False)
negative_docs = process_docs('txt_sentoken/neg', vocab, False)
test_docs = negative_docs + positive_docs
encoded_docs = tokenizer.texts_to_sequences(test_docs)

Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])


In [146]:
#taille du vocabulaire
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

25768


In [147]:
#Une configuration CNN avec 32 filtres  
#une taille de noyau de 8 
# une fonction d'activation linéaire rectifiée («relu»)

# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 1310, 32)          25632     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 655, 32)           0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 20960)             0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                209610    
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 2,812,053
Trainable params: 2,812,053
Non-trainable params: 0
___________________________________________

la couche d’incorporation attend des documents d’une longueur de 1373 mots et encode chaque mot du document en tant que vecteur de 100 éléments.

Nous utilisons une fonction de perte d'entropie croisée binaire car le problème que nous apprenons est un problème de classification binaire. La mise en œuvre efficace par Adam de la descente de gradient stochastique est utilisée et nous gardons une trace de la précision en plus de la perte pendant l'entraînement.

In [137]:

# compile réseau
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(Xtrain, ytrain, epochs=10, verbose=2)

C:\Users\Hanaa\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
 - 11s - loss: 0.6899 - accuracy: 0.5478
Epoch 2/10
 - 10s - loss: 0.5147 - accuracy: 0.7678
Epoch 3/10
 - 11s - loss: 0.0756 - accuracy: 0.9922
Epoch 4/10
 - 11s - loss: 0.0059 - accuracy: 1.0000
Epoch 5/10
 - 11s - loss: 0.0022 - accuracy: 1.0000
Epoch 6/10
 - 11s - loss: 0.0014 - accuracy: 1.0000
Epoch 7/10
 - 10s - loss: 0.0010 - accuracy: 1.0000
Epoch 8/10
 - 11s - loss: 7.5061e-04 - accuracy: 1.0000
Epoch 9/10
 - 10s - loss: 4.7058e-04 - accuracy: 1.0000
Epoch 10/10
 - 10s - loss: 3.1726e-04 - accuracy: 1.0000


In [128]:
# evaluer
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 86.000001


Nous pouvons voir que le modèle atteint très rapidement 100% de précision sur l'ensemble de données d'apprentissage.

**À la fin de l'analyse, le modèle atteint une précision de 86% sur l'ensemble de données de test, ce qui constitue un excellent score.**